<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/CB_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fse

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,callbacks
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRUCell, GRU, Dropout
from keras.models import Sequential
from pathlib import Path
from sklearn.metrics import r2_score
import re
import gzip
import matplotlib.pyplot as plt
import json
import math

In [ ]:
import seaborn as sns
import spacy
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Musical_Instruments.json.gz

--2020-10-06 05:19:13--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232750177 (222M) [application/octet-stream]
Saving to: ‘Musical_Instruments.json.gz’

Musical_Instruments 100%[===================>] 221.97M  18.7MB/s    in 13s     

2020-10-06 05:19:26 (17.1 MB/s) - ‘Musical_Instruments.json.gz’ saved [232750177/232750177]



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Musical_Instruments.json.gz


--2020-10-06 05:19:28--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170266128 (162M) [application/octet-stream]
Saving to: ‘meta_Musical_Instruments.json.gz’

meta_Musical_Instru 100%[===================>] 162.38M  18.6MB/s    in 9.7s    

2020-10-06 05:19:38 (16.7 MB/s) - ‘meta_Musical_Instruments.json.gz’ saved [170266128/170266128]



In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        d = json.loads(l)
        yield d
      
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
reviews = getDF('/content/Musical_Instruments.json.gz')

In [ ]:
len(reviews)

1512530

In [ ]:
metadata =getDF('/content/meta_Musical_Instruments.json.gz')

Prétraitement description


In [ ]:
metadata['description'].replace('', np.nan, inplace=True)
df_data = metadata.dropna()
df_data.reset_index(drop=True, inplace=True)


In [ ]:
df_data = df_data[['asin','description','title','price','image']]

In [ ]:
df_data['description'] = df_data['description'].apply(lambda x: ','.join(map(str, x )))

In [ ]:
df_data['description'].drop_duplicates(inplace=True)

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nlp = spacy.load("en_core_web_sm")
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def furnished(text):
    final_text = []
    for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)

df_data.description = df_data.description.apply(remove_html)
df_data.description = df_data.description.apply(remove_punctuation)            
df_data.description = df_data.description.apply(furnished)

In [ ]:
df_data['description'].drop_duplicates(inplace=True)

In [ ]:
df_data['description_training'] = [i.lower().replace(',', '').split() for i in df_data['description']]

In [ ]:
from gensim.models import FastText
ft = FastText(df_data['description_training'], min_count=1, size=50)

In [ ]:
from fse.models import Average
from fse import IndexedList
model = Average(ft)
model.train(IndexedList(list(df_data['description_training'])))



(110113, 6945133)

In [ ]:
model.sv.vectors[11]

array([ 2.099985  , -1.9732792 , -0.7152648 , -0.9709917 ,  1.8718619 ,
       -1.5139798 ,  0.4719567 , -0.9287318 ,  0.87873423,  0.37333512,
       -2.5944443 , -0.7714827 , -0.57486767,  1.8132571 ,  0.13054511,
        0.09551872,  1.9552333 , -0.58638555, -0.76551265,  0.65032667,
       -0.38505864,  0.49961478, -0.8723547 ,  1.4733427 ,  0.88990015,
        1.3013365 , -0.5239546 ,  1.1405053 , -1.568254  , -2.2694836 ,
        0.6150302 ,  2.112873  ,  0.27690473, -0.06236373,  0.10289153,
        0.7109703 ,  2.2072    ,  2.902244  , -0.9788824 ,  2.8931317 ,
        0.65225536,  1.3239552 ,  0.41314304,  0.65193045, -0.31616455,
        2.0108328 , -0.85842484,  1.0259336 , -0.23693942, -0.04866204],
      dtype=float32)

In [ ]:
len(model.sv.vectors)

120194

In [ ]:
def listToString(s):  
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

In [ ]:
df_data['description_training'].apply(lambda x: listToString(x))

0         new waterproof aluminum encased rigid led ligh...
1         10 greatest including hard say heart hotels le...
2         bray highly durable comfortable floral sunflow...
3                                                          
4                                                          
                                ...                        
120189    package includes 10 pc 3pdt true bypass switch...
120190    package includes 5 pc 3pdt true bypass switch ...
120191    high quality clear crystal singing bowl made 9...
120192    frog rubbing spine wooden stick motion produci...
120193    want sense security dropping guitar middle gig...
Name: description_training, Length: 120194, dtype: object

In [ ]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos[0]

In [ ]:
def recommandation(asin):
  idx = df_data[df_data['asin'] == asin].index
  list_rec_idx = []
  
  top_5 = model.sv.most_similar(idx, indexable=sentences)[1:6]
  
  for i in top_5:
    print(df_data['title'][i[1]])

In [ ]:
recommandation('9814272183')

Prétraitement du prix

In [ ]:
#On enleve le signe dollars 
df['price']=df['price'].str.slice(1,-1)

In [ ]:
#A l'aide d'une regex , nous prenons que les valeurs numériques
df['price'] = df['price'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float)

In [ ]:
#Nous sauvegardons le mix et le max pour la normalisation
max_price = df['price'].max()
min_price = df['price'].min()

In [ ]:
#Nous effectuons une normalisation sur le prix pour avoir une valeur entre 0 et 1 , 0 voulant dire pas cher du tout et 1 trés cher
df['price_normalise'] = df['price'].apply(lambda x: (x - min_price) / (max_price - min_price))

JKDQFDSQHFQSJKL


In [ ]:
y = df['overall'].values

In [ ]:
type(model.sv.vectors)

In [ ]:
#Tout d'abord , nous recuillons les users de facon unique dans une liste
user_ids = df["reviewerID"].unique().tolist()
#Nous représentons chaque utilisateur de notre liste en un nombre entier et vice versa
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

#Nous remplacons les colonnes de notre DF contenant les ID des utilisateurs et des articles sous format caractères par le format encodé en  nombre entier
df["reviewerID"] = df["reviewerID"].map(user2user_encoded)
#Nous récupérons la taille des vecteurs des utilisateurs et articles
num_users = len(user2user_encoded)


In [ ]:
df.columns

In [ ]:
df_x=df[['description_embedded','reviewerID','price_normalise']]

In [ ]:
df_x.dropna(inplace=True)

In [ ]:
x = df_x.values

In [ ]:
train_indices = int(0.7 * df.shape[0])

x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)


In [ ]:
model2 = Sequential()

model2.add(Dense(64,activation='relu'))

model2.add(Dense(32, activation='relu'))
model2.add(Dense(1,activation='linear'))
model2.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(lr=1e-2),metrics=[tf.keras.metrics.RootMeanSquaredError(),"mae"])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)



In [ ]:
model2.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(lr=1e-2),metrics=[tf.keras.metrics.RootMeanSquaredError(),"mae"])

In [ ]:
history = model2.fit(
    x_train,
    y_train,
    epochs=200,
    validation_data=(x_val,y_val),
    callbacks=[es])

In [ ]:
df = reviews.merge(metadata,on='asin',)
df.columns